In [9]:
import math
import numpy as np
import pandas as pd



In [12]:
df_fin = pd.read_csv('./pickle_files/df_final_spark_top5k_080421.csv',index_col=0)

In [13]:
df_fin.head()

,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey
0,1000012530,0.016220,99.99,2021-02-13,66.081387,-0.0187,0.0059,True,44.680,247523697279378
1,1000012794,0.014916,259.99,2021-02-25,41.525974,-0.0120,0.0052,False,141.050,151936474932722
2,1000021922,0.023490,114.99,2021-02-16,90.113095,-0.0194,0.0025,True,NaN,156995084198890
3,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,11954970507506
4,1000024320,0.024532,119.99,2020-12-01,22.354839,-0.0220,0.0097,False,65.109,31827848594342


In [14]:
df_final_spark=df_fin.astype({'pc2': 'float64'})

print(df_final_spark.dtypes)

df_final_spark.dropna(inplace=True)
df_final_spark = df_final_spark[df_final_spark['PE']< 1.0]
df_final_spark = df_final_spark[df_final_spark['PE_err']<1.0]

item_code             int64
avg_conv_rate       float64
last_price          float64
last_date            object
avg_unique_views    float64
PE                  float64
PE_err              float64
has_enough_stat        bool
pc2                 float64
item_skey             int64
dtype: object


In [15]:
VAT = 1.19



In [16]:
def create_auxiliary_var(dataf=df_final_spark):
    dataf['Delta_price'] = (dataf['last_price']/VAT - dataf['pc2'])
    dataf['Margin_base'] = dataf['avg_unique_views']*dataf['avg_conv_rate']*(dataf['Delta_price'])

def classify(dataf= df_final_spark):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_up'] - 3*dataf['Margin_err_up']), 'up',
                                        np.where(dataf['Margin_base'] < (dataf['Margin_down'] - 3*dataf['Margin_err_down']), 'down','neutral'))



In [17]:
def create_Margin( increment = 0.01, dataf = df_final_spark, flag = 'up'):
    if flag == 'down':
        increment = -increment

    price          = 'price_'+flag
    Delta_price    = 'Delta_price_'+flag
    conv_rate      = 'conv_rate_'+flag
    conv_rate_err  = 'conv_rate_err_'+flag
    Margin         = 'Margin_'+flag
    Margin_err     = 'Margin_err_'+flag

    dataf[price]              =  dataf['last_price']*math.exp(increment)
    dataf[Delta_price]        =  (dataf[price]/VAT - dataf['pc2']) #MEVI I am overwriting this variable
    dataf[conv_rate]          =  dataf['avg_conv_rate']*np.exp(dataf['PE']*np.log(dataf[price]/dataf['last_price']))
    dataf[conv_rate_err]      =  np.fabs(dataf[conv_rate])*np.fabs(dataf['PE_err']*np.log(dataf[price]/dataf['last_price']))
    dataf[Margin]             =  dataf['avg_unique_views']*dataf[conv_rate]*(dataf[Delta_price])
    #Propagate Margin error
    dataf[Margin_err]          =  dataf['avg_unique_views']*dataf[conv_rate_err]*(dataf[Delta_price])



In [18]:
print(df_final_spark[['item_code','avg_conv_rate','last_price']].head())
#create_Margin(0.01)

    item_code  avg_conv_rate  last_price
0  1000012530       0.016220       99.99
1  1000012794       0.014916      259.99
3  1000022179       0.003121      479.99
4  1000024320       0.024532      119.99
5  1000034743       0.005481      279.99


In [30]:
original_df= df_final_spark
print(len(original_df.index))
tmp_df = pd.DataFrame()

original_df_up   = pd.DataFrame()
original_df_down = pd.DataFrame()

4155


In [31]:
create_auxiliary_var(original_df)
for inc in [0.01,0.02]: #np.linspace(0.01,0.1,10):
    print('Increment = ' + str(inc))
    if inc == 0.01:
        create_Margin(inc,original_df,'up')
        create_Margin(inc,original_df,'down')
        classify(original_df)
        tmp_df           = tmp_df.append(original_df[original_df['Group_flag'] == 'neutral'])
        original_df_up   = original_df_up.append(original_df[original_df['Group_flag'] == 'up'])
        original_df_down = original_df_down.append(original_df[original_df['Group_flag'] == 'down'])
        print('tmp\n',tmp_df.head())
        print('up\n', original_df_up.head())
        print('down\n',original_df_down.head())
    if inc > 0.01:
        create_Margin(inc,original_df_up,'up')
        create_Margin(inc,original_df_down,'down')
        classify(original_df_up)
        classify(original_df_down)
        tmp_df           = tmp_df.append(original_df_up[original_df_up['Group_flag'] == 'neutral'])
        tmp_df           = tmp_df.append(original_df_down[original_df_down['Group_flag'] == 'neutral'])
        original_df_up   = original_df_up[original_df_up['Group_flag'] == 'up']
        original_df_down = original_df_down[original_df_down['Group_flag'] == 'down']
        print('tmp\n',tmp_df.head())
        print('up\n', original_df_up.head())
        print('down\n',original_df_down.head())
    original_df_down['Margin_base']   = original_df_down['Margin_down']
    original_df_up['Margin_base']   = original_df_up['Margin_up']

    print(len(tmp_df.index))
    print(len(original_df_up.index))
    print(len(original_df_down.index))

Increment = 0.01
tmp
 Empty DataFrame
Columns: [item_code, avg_conv_rate, last_price, last_date, avg_unique_views, PE, PE_err, has_enough_stat, pc2, item_skey, Delta_price, Margin_base, price_up, Delta_price_up, conv_rate_up, conv_rate_err_up, Margin_up, Margin_err_up, price_down, Delta_price_down, conv_rate_down, conv_rate_err_down, Margin_down, Margin_err_down, Group_flag]
Index: []

[0 rows x 25 columns]
up
     item_code  avg_conv_rate  last_price   last_date  avg_unique_views  \
0  1000012530       0.016220       99.99  2021-02-13         66.081387   
1  1000012794       0.014916      259.99  2021-02-25         41.525974   
3  1000022179       0.003121      479.99  2021-02-06         93.258739   
4  1000024320       0.024532      119.99  2020-12-01         22.354839   
5  1000034743       0.005481      279.99  2021-02-04        110.202494   

       PE  PE_err  has_enough_stat      pc2        item_skey  ...  \
0 -0.0187  0.0059             True   44.680  247523697279378  ...   
1 

In [ ]:
df_final_spark.head(400)


In [ ]:
tmp_df[['item_code','avg_conv_rate','last_price','PE','PE_err','has_enough_stat','pc2','price_up','price_down','Margin_base','Margin_up','Margin_err_up','Margin_down','Margin_err_down','Group_flag']].head(400)


In [ ]:
tmp_df = tmp_df.sort_index()

In [ ]:
tmp_df['Margin_last'] = tmp_df['Margin_base']
tmp_df.head()

In [ ]:
create_Margin(0.01,df_final_spark)
df_final_spark.head()

In [ ]:
tmp_df['Margin_base']=df_final_spark['Margin_base']

tmp_df['Margin_rate_to_base'] = np.log(tmp_df['Margin_down']/tmp_df['Margin_base'])
#tmp_df['Margin_rate_to_last'] = np.log(tmp_df['Margin_down']/tmp_df['Margin_last'])

In [ ]:
tmp_df[['item_code','avg_conv_rate'	,'last_price','avg_unique_views', 'price_down', 'Delta_price', 'Margin_down','Margin_base','Margin_down_err','Margin_last','Group_flag','Margin_rate_to_base']]

In [ ]:
print(tmp_df['Margin_rate_to_base'].mean()*100)

In [ ]:
tmp_df.to_csv('../raw_data/Excel_files/Margin_result_PE_analysis_PC2_no_stock_no_stat_filter_20210226.csv')


